# [TECHDOCS]Getting Started

## Introduction

This example describes how to connect to Granta MI and perform a basic Impacted Substances query. It also describes
how to view logging messages returned by the Granta MI server. It does not go into detail about the results of the
queries, please see the other examples for further detail in this area.

## Connect to Granta MI

The first step is to connect to the Granta MI server. Use the `ansys.grantami.bomanalytics.Connection` class to create
a connection. The `Connection` class uses a fluent interface to build the connection which is always invoked in the
following sequence:

1. Specify the Granta MI Service Layer URL as a parameter to the `Connection` class
2. Specify the authentication method using one of the `Connection.with_...()` methods
3. Use the `Connection.connect()` method to finalize the connection. This returns a connection object which is called
   `cxn` in these examples.

In [1]:
from ansys.grantami.bomanalytics import Connection

server_url = "http://my_grantami_server/mi_servicelayer"

If the Python script is running on Windows, in most cases you will be able to use the `.with_autologon()`.

In [2]:
cxn = Connection(server_url).with_autologon().connect()
cxn

<BomServicesClient: url="http://my_grantami_server/mi_servicelayer", dbkey="MI_Restricted_Substances">

If the Python script is running on Linux without Kerberos enabled, or you wish to use an account other than your
logged-in account, you can specify credentials explicitly.

In [3]:
cxn = Connection(server_url).with_credentials("my_username", "my_password").connect()
cxn

<BomServicesClient: url="http://my_grantami_server/mi_servicelayer", dbkey="MI_Restricted_Substances">

OIDC and anonymous authentication methods are also available, but these are beyond the scope of this example. See
the documentation for the `ansys-openapi-common` package for more details.

## Construct a Query

Queries are also constructed using a fluent interface. However, the `Query` constructor takes no arguments; all the
details of the query are specified on `Query` methods.  To demonstrate this, we will aim to build a query that
determines the substances present in an ABS material that are impacted by the REACH Candidate List legislation.

First import the `queries` module and create a `MaterialImpactedSubstancesQuery` object.

In [4]:
from ansys.grantami.bomanalytics import queries

query = queries.MaterialImpactedSubstancesQuery()
query

<MaterialImpactedSubstancesQuery: 0 materials, batch size = 100, 0 legislations>

Now add the material that we wish to query by specifying its material ID. Note: other methods of specifying records
are available; these are discussed in other examples.

In [5]:
query = query.with_material_ids(["plastic-abs-high-impact"])
query

<MaterialImpactedSubstancesQuery: 1 materials, batch size = 100, 0 legislations>

Note that because the `MaterialImpactedSubstancesQuery` object has a fluent interface, we receive the same object back
that we started with, just with the Material IDs added. Finally, add the legislation to the query.

In [6]:
query = query.with_legislations(["REACH - The Candidate List"])
query

<MaterialImpactedSubstancesQuery: 1 materials, batch size = 100, 1 legislations>

However, fluent interfaces are intended to allow a complex object to be constructed in a single line of code. We can
consolidate the above cells to create the object in a single step:

In [7]:
query = queries.MaterialImpactedSubstancesQuery().with_material_ids(["plastic-abs-high-impact"]).with_legislations(["REACH - The Candidate List"])  # noqa: E501
query

<MaterialImpactedSubstancesQuery: 1 materials, batch size = 100, 1 legislations>

Since the fluent interface can produce very long lines of code, it is required to break the query creation into
multiple lines. The following multi-line format is used throughout the examples, and is functionally equivalent to
the cell above.

In [8]:
query = (
    queries.MaterialImpactedSubstancesQuery()
    .with_material_ids(["plastic-abs-high-impact"])
    .with_legislations(["REACH - The Candidate List"])
)
query

<MaterialImpactedSubstancesQuery: 1 materials, batch size = 100, 1 legislations>

The cell above is the recommended way of creating queries using this API.

## Run a Query

Now we have our `cxn` and `query` objects, we can use the `cxn.run()` method to run the query. This returns an object
that contains the results of the query.

In [9]:
result = cxn.run(query)
result

<MaterialImpactedSubstancesQueryResult: 1 MaterialWithImpactedSubstances results>

## Query Results

In the case of an `MaterialsImpactedSubstancesQuery`, the results object contains the list of substances present in
the material that are impacted by the specified legislations.

In [10]:
result.impacted_substances

[<ImpactedSubstance: {"cas_number": "15087-24-8", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "25973-55-1", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "36437-37-3", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "3846-71-7", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "3864-99-1", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "872-50-4", "percent_amount": None}>,
 <ImpactedSubstance: {"cas_number": "90481-04-2", "percent_amount": None}>]

## Logging

All query results contain a list of messages that were returned by the server when running the query. These are
sorted in order of decreasing severity. The same messages are also available in the Service Layer log file.

In [11]:
result.messages

[]

These messages are also available via the standard `logging` module using the 'ansys.grantami.bomanalytics' logger.
Alternatively, omit the logger name to get the root logger, which will include messages logged by all packages.
The code below creates a log handler that outputs all bomanalytics messages with severity INFO and above to either the
terminal or the notebook.

In [12]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ansys.grantami.bomanalytics")

result = cxn.run(query)

INFO:ansys.grantami.bomanalytics:Running query <MaterialImpactedSubstancesQuery: 1 materials, batch size = 100, 1 legislations> with connection <BomServicesClient: url="http://my_grantami_server/mi_servicelayer", dbkey="MI_Restricted_Substances">


INFO:ansys.grantami.bomanalytics:Using default table config


INFO:ansys.grantami.bomanalytics:Using default database key (MI_Restricted_Substances)
